# Structured Streaming with Apache Kafka

## Example 1

Reading a Kafka topic in AWS.
Before executing this code, replace `kafka:9094` by the right bootstrap server

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

# Read streaming data from Kafka
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "34.139.223.68:9094") \
  .option("subscribe", "toots") \
  .load()
  
# Define the schema for parsing JSON data
schema = StructType(
    [
        StructField('id', StringType(), True),
        StructField('content', StringType(), True),
        StructField('created_at', StringType(), True),
        StructField('account', StringType(), True)
    ]
)

# Print the schema of the DataFrame
df.printSchema()

# Parse the value column from Kafka as JSON and expand it into separate columns
dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

Last parse: the entire operation converts the raw Kafka streaming data (`df`) into a more structured format (`dataset`).
1. **Initial `selectExpr` Transformation**:
   - `df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp")`: This selects and casts columns from the DataFrame `df`.
     - `"CAST(key AS STRING)"`: Casts the column `key` to a string type. This is typically done to ensure compatibility or to handle data in a specific format.
     - `"CAST(value AS STRING)"`: Casts the column `value` to a string type. In many cases, Kafka streams data as byte arrays, so casting it to a string is necessary for further processing.
     - `"timestamp"`: Presumably, this refers to the existing `timestamp` column in `df`, which is included as-is.

2. **Parsing JSON with `from_json`**:
   - `.withColumn("value", from_json("value", schema))`: This adds or replaces the `value` column by parsing its content as JSON using the specified `schema`.
     - `"value"`: Refers to the column name `value` which contains JSON data.
     - `from_json("value", schema)`: Uses the `from_json` function to parse JSON data from the `value` column according to the provided `schema`.
     - `schema`: Represents the predefined structure (`StructType`) that defines the expected fields (`StructField`) and their types in the JSON data.

3. **Final `select` Transformation**:
   - `.select(col('key'), col("timestamp"), col('value.*'))`: Selects columns from the DataFrame `df` after parsing JSON.
     - `col('key')`: Selects the `key` column as-is.
     - `col("timestamp")`: Selects the `timestamp` column as-is.
     - `col('value.*')`: Selects all columns (`*`) from the `value` struct column that was parsed from JSON in the previous step. This expands the struct column into individual columns.


In [0]:
# Specify the output mode as 'append' (only new rows added to the result table)
# Define the output sink format as 'memory' (store the result in-memory table)
# Option to truncate long strings in the output table (set to 'false' to display full content)
# Assign a query name for the streaming query (to be referenced in Spark SQL)
# Start the streaming query
dataset.writeStream \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("toots_topic") \
 .start()

The above block of code sets up a streaming pipeline in Spark Structured Streaming that reads data (`dataset`), processes it in append mode, stores the results in-memory, ensures full string content is displayed, assigns a query name, and starts the streaming execution. `writeStream` is a method used to define how the streaming data should be written to an external sink in a continuous and streaming manner. 

It enables real-time data processing and analysis, making the results immediately available for querying using Spark SQL or other downstream applications.

In [0]:
%sql
SELECT
  *
FROM
  toots_topic

## Exercise 1 - Sliding window

Apply a sliding window each minute, 5 minutes of duration, grouping by `server`. A server in Mastodon is the domain in account column

---



In [0]:
from pyspark.sql.functions import col, window, split, when

# Displaying grouped and counted data
#    Extracts the server part from 'account' column if it contains '@'
#    Groups data by a 5-minute sliding window and 'server'
#    Counts occurrences within each window and server group
#    Displays the result
(
dataset
  .withColumn("server", when(col("account").contains("@"), split(col("account"), "@").getItem(1))
                   .otherwise(None))
  .groupBy(window(col("timestamp"), "5 minutes", "1 minutes"), col("server"))
  .count()
  .display()
)

In [0]:
from pyspark.sql.functions import col, window, split, when

# Writing streaming data with update mode
#    Extracts the server part from 'account' column if it contains '@'
#    Groups data by a 5-minute sliding window and 'server'
#    Counts occurrences within each window and server group
#    Specifies update mode for streaming write
#    Sets the output sink format to in-memory table
#    Ensures full content display in the in-memory table
#    Assigns a name to the streaming query
#    Starts the streaming query
(
dataset
  .withColumn("server", when(col("account").contains("@"), split(col("account"), "@").getItem(1))
                   .otherwise(None))
  .groupBy(window(col("timestamp"), "5 minutes", "1 minutes"), col("server"))
  .count()
  .writeStream \
  .outputMode("update") \
  .format("memory") \
  .option("truncate", "false") \
  .queryName("toots_update_topic") \
  .start()
)

In [0]:
%sql
select * from toots_update_topic

## Exercise 2 - Get the last 5 min number of toots each minute

Each minute, get the number of toots received in last 5 minutes

In [0]:
# Writing windowed data with update mode
#    Groups data by a 5-minute sliding window
#    ounts occurrences within each window
#    Specifies update mode for streaming write
#    Sets the output sink format to in-memory table
#    Assigns a name to the streaming query
#    Starts the streaming query
dataset.groupBy(window(col("timestamp"), "5 minutes", "1 minutes")) \
    .count() \
    .writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName("toots_windowed_2") \
    .start()

In [0]:
%sql
SELECT
  *
FROM
  toots_windowed_2
ORDER BY
  count DESC

## Exercise 3 - Get top words in 1 min slots

Get top words with more than 3 letters in 1 minute slots


In [0]:
from pyspark.sql.functions import lower, explode, length

# Displaying word counts within 1-minute windows
#    Selects columns key, value, and timestamp, casting key and value as strings
#    Parses the value column from JSON format using the specified schema
#    Selects key, timestamp, and all columns from the parsed JSON value
#    Splits the content column by spaces, then explodes it into rows of words, preserving the timestamp
#    Filters out words with a length less than or equal to 3
dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*')) \
    .select(explode(split(col("content"), " ")).alias("word"), "timestamp") \
    .filter(length(col("word")) > 3) # Filters out words with length less than or equal to 3

# Groups data by a 1-minute sliding window and 'word'
# Counts occurrences of each word within each window and displays the result
dataset.groupBy(window(col("timestamp"), "1 minutes"), col("word")) \
    .count() \
    .display()

## Clean up DBFS

In [0]:
%scala
// Clean up
val PATH = "dbfs:/tmp/" // Define the base path for cleanup

// List all files and directories under the specified path
// Extract only the names of the files and directories
// Iterate through each file and delete it recursively
dbutils.fs.ls(PATH)
            .map(_.name)
            .foreach((file: String) => dbutils.fs.rm(PATH + file, true))